In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import precision_score, recall_score

# Load the data

In [2]:
# Split Movielens 100K data into train and test (80-20)
np.seed = 1
dataset = pd.read_csv("../data/u.data",sep='\t',names="user_id,item_id,rating,timestamp".split(","))
dataset = dataset.iloc[:,:3]
dataset.user_id = dataset.user_id.astype('category').cat.codes.values
dataset.item_id = dataset.item_id.astype('category').cat.codes.values
train, test = train_test_split(dataset, test_size=0.2)

In [4]:
# Check that we have ratings in the train set for all the users in the test set
for test_user in test["user_id"].values:
    if test_user not in train["user_id"].values:
        print("User", test_user, "is in the test set but not in the train set")

# MostPop Recommender

In [64]:
# Get the items with the most interactions (ratings)
item_counts = train.item_id.value_counts()

# define mostpop algorithm
def mostpop(uid, k=5):
    return item_counts.index[:k].values.tolist()

In [11]:
# Compute similarities

# First create the user-item matrix
unique_users = dataset.user_id.unique()
unique_items = dataset.item_id.unique()
ui_matrix = np.zeros((unique_users.shape[0], unique_items.shape[0]))

# Use train data to build the similarity matrix
for train_row in train.itertuples():
    ui_matrix[train_row.user_id - 1, train_row.item_id - 1] = train_row.rating

print(ui_matrix.shape) # We should have a 943x1682 matrix

# Item-User matrix is the transpose of the User-Item matrix
iu_matrix = ui_matrix.T
print(iu_matrix.shape) # We should have a 1682x943 matrix

(943, 1682)
(1682, 943)


# Evaluation Functions

In [47]:
# First of all construct the dictionary containing the ground truth rankings
test_gt_ranks = {}
for uid in test.user_id:
    user_items = test[test.user_id == uid]
    user_items = user_items[user_items.rating > 3.5]
    user_items = user_items.sort_values(by=["rating"], ascending=False)
    test_gt_ranks[uid] = user_items.item_id.values.tolist()

{662: [186, 55, 271, 133, 173, 126, 149, 95, 49, 1244, 627, 474, 120, 6, 741, 244, 1326, 182, 947], 762: [134, 99, 55, 197, 1064, 68, 4, 38, 172, 844, 189], 456: [146, 96, 195, 193, 11, 150, 213, 68, 182, 631, 194, 190, 189, 191, 530, 672, 231, 472, 844, 384, 10, 181, 124, 567, 366, 218, 840, 52, 365, 470, 55, 154, 355], 77: [300, 410], 757: [895, 482, 167, 1141, 126, 481, 508, 134, 175, 176, 7, 513, 383, 473, 136, 606, 1097, 1006, 344, 704, 285, 519, 478, 222, 153, 151, 237, 199, 152, 411, 511, 95, 1158, 284, 1015, 49, 352, 233, 25, 351, 528, 653, 146, 27, 525, 318, 337, 341, 586, 127, 714, 302, 501, 863, 830, 566, 461, 789, 507], 781: [872, 287, 751, 243, 1390, 321, 537, 349, 886, 347], 434: [183, 167, 461, 272, 173, 194, 0, 81, 11, 176, 55, 178, 317, 762, 171, 1068, 658, 608, 78, 745, 741, 21, 90, 1043, 1102, 85, 193, 126, 201, 6, 233, 48, 155, 761, 651, 777, 209, 750, 636, 639, 190], 193: [63, 465, 14, 0, 210, 12, 736, 514, 836, 528, 70, 203, 518, 293, 187, 160, 156, 264, 123, 431,

In [59]:
# Check that we have ratings in the train set for all the users in the test set
def find_items_not_in_trainset(trainset, testset):
    items_not_in_train = []
    for itemId in testset.item_id.values:    
        if itemId not in trainset.item_id.values:
            items_not_in_train.append(itemId)
            
    return items_not_in_train

# This method just gets all the items the user has rated in the trainset        
def user_seen_items(userId):
    return train[train.user_id == userId].item_id.index.values.tolist()

In [ ]:
def precision_recall_at_k(predictions, items_not_in_train, k, threshold=3.5):
    top_n_recoms_est = GetTopN(predictions, n=k, minimumRating=threshold, criterion="est")
    top_n_recoms_real = GetTopN(predictions, n=k, minimumRating=threshold, criterion="r_ui")
    above_threshold = predictions[predictions.r_ui >= threshold]

    precisions = {}
    recalls = {}

    for uid, est_topn in top_n_recoms_est.items():
        # Get items the user has already rated
        already_seen = user_seen_items(uid)
        # Get relevant items for the user
        n_rel_for_user = len(above_threshold[above_threshold.uid == uid])        
        tp = 0
        # Penalize the scores if:
        # - The item we are recommending was never seen in the training set (how could we recommend what we don't know?)
        # - The user has already rated this item: It's not a good recommendation since the user already knows it/has seen it
        for est_itemId, _ in est_topn:
            if(est_itemId in items_not_in_train or est_itemId in already_seen):
                tp += 0
            else:
                for real_itemId, _ in top_n_recoms_real[uid]:
                    if (est_itemId == real_itemId):
                        tp +=1
        
        precisions[uid] = tp/k
        recalls[uid] = tp/n_rel_for_user if n_rel_for_user != 0 else 0

    return precisions, recalls

In [80]:
#def calc_precision
k=5
total_precisions = 0

for uid in test.user_id:
    gt_rank = test_gt_ranks[uid][:k]
    predicted = mostpop(uid, k)
    if (len(predicted) > len(gt_rank)):
        predicted = predicted[:len(gt_rank)]
        
    total_precisions += precision_score(gt_rank, predicted, average='weighted')
    

/Users/Geri/.local/share/virtualenvs/recsys2020_project-EWQ0OJVi/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
total_prec_sum / len(test.user_id)

0.008211845238095088

In [82]:
total_precisions/940

0.19264184397162992